In [ ]:
import os
import shutil

# 1) Get the parent directory (where this script and 'otherDatasets' live)
PARENT_DIR = os.getcwd()
OTHER_DIR  = os.path.join(PARENT_DIR, "otherDatasets")
MERGED_DIR = os.path.join(PARENT_DIR, "mergedData")

# 2) Find all dataset subfolders under 'otherDatasets', skipping any mergedData
DATASETS = [
    d for d in os.listdir(OTHER_DIR)
    if os.path.isdir(os.path.join(OTHER_DIR, d))
    and d.lower() != "mergeddata"
]

# 3) Create the 'mergedData' folder at the same level as 'otherDatasets'
os.makedirs(MERGED_DIR, exist_ok=True)

for ds in DATASETS:
    ds_path    = os.path.join(OTHER_DIR, ds)
    train_path = os.path.join(ds_path, "train")

    # If there's a train/ folder, only process that; otherwise process the dataset root.
    roots = [train_path] if os.path.isdir(train_path) else [ds_path]

    for root in roots:
        for folder in os.listdir(root):
            folder_src = os.path.join(root, folder)
            if not os.path.isdir(folder_src):
                continue

            # Normalize to uppercase
            folder_name = folder.upper()
            folder_dst  = os.path.join(MERGED_DIR, folder_name)
            os.makedirs(folder_dst, exist_ok=True)

            # Copy each file into mergedData/<LETTER>/,
            # appending the dataset name to the filename
            for fname in os.listdir(folder_src):
                src_file = os.path.join(folder_src, fname)
                if not os.path.isfile(src_file):
                    continue  # skip subfolders, etc.

                base, ext = os.path.splitext(fname)
                new_fname = f"{base}_{ds}{ext}"
                dst_file  = os.path.join(folder_dst, new_fname)

                # Skip duplicates if they somehow collide
                if os.path.exists(dst_file):
                    print(f"Skipping duplicate: {dst_file}")
                    continue

                shutil.copy2(src_file, dst_file)
